# How to use RUPsycho Parsers

Welcome to the introductory guide for the RUPsycho Python package. This guide aims to provide a comprehensive overview of RUPsycho, a new and innovative software tool developed for the application of large language models (LLMs) in social science research. The RUPsycho package is designed to facilitate the exploration of humanlike behaviors through LLMs, offering a unique perspective in the field of natural language processing (NLP).

This guide will walk you through the various components and functionalities of RUPsycho. You will learn how to set up the package, navigate its testing environments, and effectively utilize its features for analyzing LLMs in the context of social science. The guide aims to equip researchers with the knowledge and skills required to leverage RUPsycho in their studies, contributing to a deeper understanding of the complex interplay between artificial intelligence and human psychology.

In [1]:
import rupsycho as rup
from rupsycho.parsers.cleaners import BasicCleaner, PromptRemovalCleaner 
from rupsycho.parsers.judges import MultipleChoiceParser
from rupsycho.parsers.validators import ApologiesValidatorParser, BeingAiValidatorParser, RefusalValidatorParser, ValidatorParser

/home/jschelb/.pyenv/versions/3.10.8/envs/rup/lib/python3.10/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


## Cleaners

This module defines a specialized parser class called Cleaner, designed to
process and clean textual answers by removing artifacts that may have been
introduced during the text generation process.

In [2]:
# Instantiate the custom parser
basic_parser = BasicCleaner()

# Example text to parse
raw_output = "Hello, world!\nThis is a test text with some emojis 😊 and line breaks.\n"

# Parse the output
parsed_output = basic_parser.parse(raw_output)

# Print the cleaned output
print("Cleaned output:", parsed_output)

Cleaned output: Hello, world! This is a test text with some emojis and line breaks.


In [3]:
prompt_example = "Once upon a time, there was a brave knight."
completion_example = "Once upon a time, there was a brave knight. He fought valiantly against the dragon."

# Instantiate the PromptRemovalCleanerParser
prompt_cleaner_parser = PromptRemovalCleaner(prompt=prompt_example)

# Parse the completion to clean it from the prompt
cleaned_completion = prompt_cleaner_parser.parse(completion_example)
print(f'Cleaned completion: {cleaned_completion}')

Cleaned completion: {'completion': 'once upon a time, there was a brave knight. he fought valiantly against the dragon.', 'similarity_score': 0.683, 'similarity_threshold': 0.7}


## Validators

This module defines a specialized parser class called Validator, designed to
assess and validate textual answers. The Validator parser takes an answer as
input and returns the original answer along with a validation status that
indicates whether the input meets specified criteria or rules.

In [4]:
# Example texts to validate
example_texts = {
    "general": "I think I would choose option 1 because it seems the best. Also, I somewhat disagree with option 2.",
    "apologies": "I apologize for the inconvenience.",
    "being_ai": "As an AI, I do not have personal opinions.",
    "refusal": "I cannot provide that information.",
    "combined": "I apologize for any confusion, but as an AI, I cannot provide that information."
}

In [5]:
# Instantiate the individual validators
apologies_parser = ApologiesValidatorParser()
being_ai_parser = BeingAiValidatorParser()
refusal_parser = RefusalValidatorParser()

# Function to validate text with a specific parser
def validate_text(parser, text_key):
    text = example_texts[text_key]
    result = parser.parse(text)
    print(f'{parser._type.capitalize()} validation result: {result}\n')

# Validate using individual parsers
print("Running individual validators:")
validate_text(apologies_parser, "apologies")
validate_text(being_ai_parser, "being_ai")
validate_text(refusal_parser, "refusal")

Running individual validators:
Apologies_validator_parser validation result: {'text': 'I apologize for the inconvenience.', 'validation_status': 'invalid', 'details': {'apologies': True}}

Being_ai_validator_parser validation result: {'text': 'As an AI, I do not have personal opinions.', 'validation_status': 'invalid', 'details': {'being_ai': True}}

Refusal_validator_parser validation result: {'text': 'I cannot provide that information.', 'validation_status': 'invalid', 'details': {'refusal': True}}



In [6]:
# Instantiate the combined ValidatorParser
validator_parser = ValidatorParser()

# Validate using the combined parser
print("Running combined validator:")
combined_result = validator_parser.parse(example_texts["combined"])
print(f'Combined validation result: {combined_result}')

Running combined validator:
Combined validation result: {'text': 'I apologize for any confusion, but as an AI, I cannot provide that information.', 'validation_status': 'invalid', 'details': {'apologies': True, 'being_ai': True, 'refusal': True}}


## Judges

This module defines a specialized parser class called Judges, designed to
evaluate and process input answers. The Judges parser takes in a textual
answer and applies a series of checks and transformations to determine a
final prediction or verdict.

In [7]:
# Define possible answers
possible_answers_1 = ["1. strongly disagree",
                        "2. somewhat agree", "3. agree"]
possible_answers_2 = ["A. strongly disagree",
                        "B. somewhat agree", "C. agree"]

# Instantiate the custom parser
parser_1 = MultipleChoiceParser(possible_answers_1)
parser_2 = MultipleChoiceParser(possible_answers_2)

# Example text to parse
text = "I think I would choose option 1 because it seems the best. Also, I somewhat disagree with option 2."

# Parse the output using both sets of possible answers
result_1 = parser_1.invoke(text)
print(f'Most likely answer with possible answers 1: {result_1}')

result_2 = parser_2.invoke(text)
print(f'Most likely answer with possible answers 2: {result_2}')

Most likely answer with possible answers 1: 1. strongly disagree
Most likely answer with possible answers 2: A. strongly disagree
